In [1]:
include("/home/kwat/github/Kraft.jl/src/Kraft.jl")

Main.Kraft

In [4]:
const genes = readlines("/home/kwat/github/kraft/notebook/genes.txt")[2:end];

const scores = fill(
    1 / length(genes),
    length(genes),
);

const gene_set_dict = Kraft.read_gmt("/home/kwat/garden/data/gene_set/msigdb_v6.2/h.all.v6.2.symbols.gmt")

const gene_set_genes = sort(gene_set_dict[sort(collect(keys(gene_set_dict)))[1]]);

In [5]:
using BenchmarkTools

In [8]:
const gene_rank = Dict(zip(
    genes,
    1:length(genes),
))

const hits = Kraft.make_hits(
    genes,
    gene_set_genes,
);

In [10]:
function compute_gene_set_enrichment(
    genes::Array{String, 1},
    scores::Array{Float64, 1},
    gene_set_genes::Array{String, 1},
    hits::Array{Int64, 1},
)
    
    # 1ns
    n = length(genes)
    
    # 2us
    cumulative_sum = Array{Float64, 1}(
        undef,
        n,
    )
    
    # 13us
    hit_scores_sum = Kraft.sum_hit_scores(
        scores,
        hits,
    )
    
    # 6us
    d_down = -1 / (n - Kraft.sum_hits(hits))
    
    # 0.01ns
    value = 0.0
    
    # 0.01ns
    auc = 0.0
    
    # 0.01ns
    min_ = 0.0
    
    # 0.01ns
    max_ = 0.0
    
    @inbounds @fastmath @simd for i in 1:n
        
        if hits[i] == 1
            
            value += abs(scores[i]) / hit_scores_sum
            
        else
            
            value += d_down
            
        end
        
        # 50us
        cumulative_sum[i] = value
        
        auc += value
        
        if value < min_
            
            min_ = value
            
        elseif max_ < value
            
            max_ = value
            
        end
            
    end
    
    cumulative_sum, auc, min_, max_
    
end


@benchmark compute_gene_set_enrichment(
    genes,
    scores,
    gene_set_genes,
    hits,
)

BenchmarkTools.Trial: 
  memory estimate:  452.94 KiB
  allocs estimate:  3
  --------------
  minimum time:     97.329 μs (0.00% GC)
  median time:      102.230 μs (0.00% GC)
  mean time:        112.929 μs (6.47% GC)
  maximum time:     31.115 ms (99.38% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [11]:
using PyCall

In [12]:
pd = pyimport("pandas")

kraft = pyimport("kraft")

PyObject <module 'kraft' from '/home/kwat/github/kraft/kraft/__init__.py'>

In [13]:
gene_score = pd.Series(
    scores,
    index=genes,
)

PyObject MIR628        0.000017
RNU6-871P     0.000017
MIR626        0.000017
AC012314.7    0.000017
GHRLOS        0.000017
                ...   
KP420441.5    0.000017
KP420440.2    0.000017
KP420440.6    0.000017
KP420440.5    0.000017
KP420446.1    0.000017
Length: 57954, dtype: float64

In [14]:
println(kraft.run_single_sample_gsea(
    gene_score,
    gene_set_genes,
    hit=hits,
    plot=false,
))

@benchmark kraft.run_single_sample_gsea(
    gene_score,
    gene_set_genes,
    hit=hits,
    plot=false,
)

-0.11891713503802033


BenchmarkTools.Trial: 
  memory estimate:  5.94 KiB
  allocs estimate:  258
  --------------
  minimum time:     508.319 μs (0.00% GC)
  median time:      517.199 μs (0.00% GC)
  mean time:        532.771 μs (0.53% GC)
  maximum time:     56.392 ms (47.14% GC)
  --------------
  samples:          9338
  evals/sample:     1